In [5]:
import pandas as pd
import re

df = pd.read_csv("일본_경제산업성_대외뉴스.csv")
df_keyword = pd.read_csv("일본_번역.csv", index_col = False)

In [12]:
import numpy as np
import itertools
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/114 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.38k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/452 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/341 [00:00<?, ?B/s]

In [13]:
keyword = list(df_keyword["번역"])
print(keyword)

['半導体', 'プロセスとコントローラー', '金属くず', '他のプラスチック製品', '鉄と非合金', '鉄と非比較可能な川', 'その他の精密化学物質', 'シリコンウェーハ', '白金', 'その他の化学産業製品', '貨物船', '混合キシレン', '食事を修正しました', '他の石油化学製品', '自動車部品', '印刷回路', 'その他の光学機器部品', 'カーフイオイル', 'Steelene', 'バルブ', 'ワニス', 'ゴルフ', '銅', 'その他の細かい化学物質', 'スラブ', '他の写真映画', '他の鋳鉄', '薬', 'その他の概要', 'その他の機械', '潤滑剤', 'ポリエチレンテレフタレートフィルム', '金', '花火', '他のガラス製品', '鉄筋', '原油', 'Mu -Gye Mokgang', '光レンズ', '輸送機', '電子顕微鏡', '他の乗用車', '顔料', '合意', '金属', 'ビレット', 'トルエン', 'その他のコレクション回路半導体', '他の機械的要素', 'その他', 'ベアリング', '他の種この製品', 'パーツを接続します', '他のコンピューター部品', 'その他の電子アプリケーションデバイス', 'その他の建設装置', '他のタイルや陶器製品', '半導体機器部品', '他のフレーズ', '液体ポンプ', '他の金属光', '鉄と非補償', '光沢', 'エチレンポリマー', '変態', 'トランジスタ', 'ステンレスおよび合金鋼線', '不思議な部分', 'ExchangeおよびDCモーター', '他の係留部品', '合金リバーコールドローリングスチールプレート', 'ステンレスおよび合金鋼のボンガン', '前例のない映画', '他のアルミニウム製品', '鋼', 'プロピレンポリマー', 'パンプパーツ', '分析テスター部品', '楽器', 'シリコン樹脂', '電気軸とギア', 'ガスポンプ', '動的', '鉄と非ハプガム川H字型の鋼', 'X -Rayおよび放射デバイスの部品', '他の光学系', '板ガラス', '鉄と非ガンガンL字型の講義', 'ニッケル', '自動コントローラー', 'エンターテイメント', 'その他の動機', '書き込み楽器', 

In [14]:
keyword_embedding = []

for ele in keyword:
    keyword_embedding.append(model.encode(ele))

In [118]:
# not_necessary = ["dress", "notice", "Others", "Other", "hat", "rice", "feed", "Agreement"]
# for index, row in df.iterrows():
#     voca_list = []
#     for voca_index, voca_row in df_keyword.iterrows():
#         if str(row["text"]).find(' '+voca_row["번역"])>0 and voca_row["번역"] not in not_necessary:
#             voca_list.append(voca_row["번역"])
#             p = re.compile(voca_row["번역"])
#             result = p.findall(str(row["text"]))
#         if result != []:
#             voca_list.append(result)
#     voca_list = sum(voca_list, [])
#     if voca_list != []:
#         print(index, set(voca_list))

In [119]:
# text = df.loc[24]["text"].replace('\n', ' ')

In [15]:
text = "[' 2022年8月1日  対外経済 萩生田経済産業大臣は、7月27日（水曜日）から7月31日（日曜日）にかけて、米国（サンフランシスコ、ワシントンDC）を訪問しました。サンフランシスコでは、シリコンバレーを訪問し、日米のスタートアップ関係者と意見交換を行いました。また、ワシントンDCでは、日米経済協議委員会（経済版２＋２）閣僚会合に出席したほか、関係閣僚との個別会談を実施しました。 1．サンフランシスコ  世界有数のスタートアップ集積地であるサンフランシスコ・シリコンバレーを訪問し、スタートアップ関係者等と意見交換を行いました。  （1）スタンフォード大学訪問  同大学構内にて、医療現場のニーズを出発点とし問題解決策の開発等を行うByers Center for Biodesignと、アカデミアと産業の橋渡しを行うSPARKプログラムについて、それぞれ関係者と意見交換しました。  \xa0 \xa0   （2）Google 本社訪問  Google本社にて、同社シニアバイスプレジテントのプラバッカ－・ラガバン氏と面会し、スタートアップの育成やイノベーションを起こす環境づくり等について意見交換を行いました。  \xa0 \xa0   （3）スタートアップ関係者との意見交換  シリコンバレーで活躍する投資家や起業家とスタートアップの現状と課題、日本のスタートアップ・エコシステム強化の方策などについて意見交換を行いました。   2．ワシントンDC （1）日米経済政策協議委員会（経済版2＋2）閣僚会合  林外務大臣、レモンド商務長官、ブリンケン国務長官とともに、日米経済政策協議委員会（経済版２＋２）閣僚会合に出席しました。   詳細は こちら をご参照ください。   共同声明・行動計画（原文） 共同声明・行動計画（仮訳） （2）レモンド商務長官との会談  レモンド商務長官と会談を行い、経済版２＋２の成果を確認するとともに、半導体協力、インド太平洋経済枠組み（IPEF）などについて議論しました。   （3）タイ通商代表との会談  タイ通商代表と会談を行い、市場歪曲的措置に連携して対処していくことを確認しました。 また、インド太平洋経済枠組み（IPEF）やビジネスと人権などについて議論しました。  ']"

In [121]:
# 단어임베딩해서 키워드에 대한 벡터표현을 뽑아내서 유사도가 높은거뽑기 0.5이상

In [122]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence
word_tokens = text_to_word_sequence(text)
word_tokens = [x for x in word_tokens if len(x)>2]

In [123]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\yoona\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [124]:
result = []
for w in word_tokens: 
    if w not in stop_words: 
        result.append(w)

In [125]:
result

['ﬁnal',
 'rule',
 'amends',
 'department',
 'homelandsecurity',
 'dhs',
 'regulations',
 'relating',
 'temporary',
 'entry',
 'cana',
 'dian',
 'mexican',
 'citizen',
 'business',
 'persons',
 'united',
 'states',
 'byreplacing',
 'references',
 'north',
 'american',
 'free',
 'trade',
 'agreement',
 'nafta',
 'references',
 'agreement',
 'unitedstates',
 'america',
 'united',
 'mexican',
 'states',
 'canada',
 'usmca',
 'usmca',
 'superseded',
 'nafta',
 'related',
 'provisions',
 'july',
 '2020',
 'chapter',
 'usmca',
 'generally',
 'maintains',
 'treat',
 'ment',
 'provided',
 'nafta',
 'respect',
 'temporary',
 'entryof',
 'canadian',
 'mexican',
 'citizen',
 'business',
 'persons',
 'substantivechanges',
 'regulations',
 'needed',
 'document',
 'simplyupdates',
 'relevant',
 'regulations',
 'replace',
 'references',
 'nafta',
 'including',
 'references',
 'appendices',
 'annexes',
 'corre',
 'sponding',
 'usmca',
 'references',
 'document',
 'also',
 'makes',
 'minor',
 'non',
 '

In [126]:
doc_embedding = model.encode(result)

In [128]:
final_result = []
for i in range(len(doc_embedding)):
    for j in range(len(keyword_embedding)):
        distances = cosine_similarity([doc_embedding[i]], [keyword_embedding[j]])
        if distances[0][0]>0.8:
            final_result.append(result[i])
# distances = cosine_similarity([doc_embedding[0]], candidate_embeddings)
print(set(final_result))

{'united', 'nafta', 'agreement', 'sponding'}
